In [1]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from pandas_ods_reader import read_ods
import pandas as pd
from scoping.models import *
import random
import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC

import matplotlib.pyplot as plt

django.db.connection.close()

In [34]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('../literature_identification/jupyter-google-e4b72bf44320.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1o18FIQTmGSWJ_m1ePAzrcVhFlfesESCb2bIEG7ibJAA'
wb = gc.open_by_key(spreadsheet_key)
ws = wb.worksheet("category_label_numbers")

table = ws.get_all_values()
category_map = pd.DataFrame(table[1:], columns=table[0])
category_map.head()



In [39]:
seen_df = pd.read_csv('../data/0_labelled_documents.csv')

df = (seen_df
      .sort_values('id')
      .sample(frac=1, random_state=1)
      .reset_index(drop=True)
)

from alterations import postfix_data
df = postfix_data(df)

print(seen_df.shape)

seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index
new_index = df[(df['seen']==1) & (df['ar5']==0)].index
rel_index = df[df['relevant']==1].index
r_index = df[df["random_sample"]==1].index
physical_index = df[df['physical_tags']==1].index



new_rel_index = physical_index.intersection(rel_index)


impact_columns = [x for x in df.columns if re.match("^4 - ",x)]
for x in impact_columns:
    row = category_map.loc[category_map['Specific category']==x.split(' - ')[1]]
    if row.shape[0]==0:
        continue
    for i, aggscheme in enumerate(row[['Proposed Aggregation 1', 'Proposed Aggregation 2']].values[0]):
        if aggscheme not in ['-','']:
            nc = f'{17+i} - {aggscheme}'
            if nc not in df.columns:
                df[nc] = 0
            df.loc[df[x]==1,nc] = 1


specific_impacts_2 = [x for x in df.columns if "18 - " in x and df[x].sum() > 5]

(2335, 242)


In [16]:
revectorize = True

X_exists = os.path.isfile(f'../data/X_{seen_df.shape[0]}.npz')

if revectorize is True or X_exists is False:
    print("running vectorisation again")
    vec = TfidfVectorizer(
        ngram_range=(1,2),
        min_df=5, max_df=0.8, strip_accents='unicode', 
        max_features=50000,
        #tokenizer=snowball_stemmer()
    )

    X = vec.fit_transform(df['content'].astype("str"))   
    with open (f'../data/vec_{seen_df.shape[0]}.pickle','wb') as f:
        pickle.dump(vec, f)
    import scipy.sparse
    scipy.sparse.save_npz(f'../data/X_{seen_df.shape[0]}.npz', X)
else:
    print("loading feature matrix")
    with open (f'../data/vec_{seen_df.shape[0]}.pickle','rb') as f:
        vec = pickle.load(f)
        X = scipy.sparse.load_npz(f'../data/X_{seen_df.shape[0]}.npz')
        
X.shape

running vectorisation again


(2335, 20676)

In [107]:
y = np.matrix(df[specific_impacts_2])

weights = {}
for i,c in enumerate(specific_impacts_2):
    weights[i] =  round((1-y[:,i].sum()/len(new_rel_index))*10)
    
weights

{0: 8.0,
 1: 9.0,
 2: 9.0,
 3: 9.0,
 4: 9.0,
 5: 10.0,
 6: 7.0,
 7: 10.0,
 8: 10.0,
 9: 9.0,
 10: 10.0,
 11: 10.0}

In [112]:
from sklearn.utils.class_weight import compute_class_weight

weights = compute_class_weight("balanced",len(specific_impacts_2),df.loc[new_rel_index,specific_impacts_2])

/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=12, y=      18 - Terrestrial Water Cycle  18 - Drought & Floods  18 - Cryosphere  \
0                                1                      0                0   
2                                0                      0                0   
5                                0                      1                0   
6                                0                      0                0   
13                               0                      0                0   
...                            ...                    ...              ...   
2326                             0                      0                0   
2327                             0                      0                0   
2329                             1                      0                0   
2332                             0                      0                0   
2333     

TypeError: 'int' object is not iterable

In [88]:
specific_impacts_2

['18 - Terrestrial Water Cycle',
 '18 - Drought & Floods',
 '18 - Cryosphere',
 '18 - Coastal and marine Ecosystems',
 '18 - Food/Agriculture',
 '18 - Carbon Cycle',
 '18 - Terrestrial ES',
 '18 - Wildfires',
 '18 - Livelihoods and wellbeing',
 '18 - Health',
 '18 - Displacement and migration',
 '18 - Other anthropogenic']

In [108]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from sklearn.model_selection import KFold


def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(100, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# test_index = random.sample(set(new_rel_index),200)
# train_index = list(set(new_rel_index) - set(test_index))

model = get_model(X.shape[1],y.shape[1])
model.fit(X[train_index], y[train_index],epochs=40, class_weight=weights.values())



Epoch 1/40
691/691 [==============================] - 1s 2ms/step - loss: 0.6278
Epoch 2/40
691/691 [==============================] - 1s 983us/step - loss: 0.4242
Epoch 3/40
691/691 [==============================] - 1s 919us/step - loss: 0.2744
Epoch 4/40
691/691 [==============================] - 1s 928us/step - loss: 0.2142
Epoch 5/40
691/691 [==============================] - 1s 921us/step - loss: 0.1800
Epoch 6/40
691/691 [==============================] - 1s 925us/step - loss: 0.1513
Epoch 7/40
691/691 [==============================] - 1s 960us/step - loss: 0.1260
Epoch 8/40
691/691 [==============================] - 1s 936us/step - loss: 0.1039
Epoch 9/40
691/691 [==============================] - 1s 895us/step - loss: 0.0855
Epoch 10/40
691/691 [==============================] - 1s 927us/step - loss: 0.0705
Epoch 11/40
691/691 [==============================] - 1s 934us/step - loss: 0.0584
Epoch 12/40
691/691 [==============================] - 1s 947us/step - loss: 0.0486
Epo

In [109]:
from sklearn.metrics import precision_score, recall_score, r2_score, f1_score
y_pred = model.predict(X[test_index])

f1scores = []

for i,c in enumerate(specific_impacts_2):
    preds = y_pred[:,i]
    f1scores.append(f1_score(y[test_index,i],preds.round()))
    
print(f1scores)
print(np.mean(f1scores))
print(np.median(f1scores))


[0.5, 0.11764705882352941, 0.5925925925925926, 0.15384615384615385, 0.18181818181818182, 0.0, 0.761904761904762, 0.25, 0.0, 0.8717948717948718, 0.0, 0.0]
0.28580030173167426
0.16783216783216784


/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [94]:
from sklearn.metrics import precision_score, recall_score, r2_score, f1_score
y_pred = model.predict(X[test_index])

f1scores = []

for i,c in enumerate(specific_impacts_2):
    preds = y_pred[:,i]
    f1scores.append(f1_score(y[test_index,i],preds.round()))
    
print(f1scores)
print(np.mean(f1scores))
print(np.median(f1scores))


[0.4615384615384615, 0.11764705882352941, 0.4347826086956522, 0.0, 0.18181818181818182, 0.0, 0.761904761904762, 0.25, 0.0, 0.8421052631578947, 0.0, 0.0]
0.2541496946615401
0.1497326203208556


/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [21]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
 

# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# make a prediction on the test set
		yhat = model.predict(X_test)
		# round probabilities to class labels
		yhat = yhat.round()
		# calculate accuracy
		acc = accuracy_score(y_test, yhat)
		# store result
		print('>%.3f' % acc)
		results.append(acc)
	return results

y = np.matrix(df[specific_impacts_2])
 
# evaluate model
results = evaluate_model(X[new_rel_index], y[new_rel_index])
# summarize performance
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



AttributeError: 'Tensor' object has no attribute 'assign'